In [96]:
%pip install requests
%pip install pandas
%pip install plotly
%pip install osmnx
%pip install folium
%pip install scipy scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
import requests as rq
import os
import json
import pandas as pd
import IPython
import plotly.express as px
import osmnx as ox
import folium
import geopandas as gpd

DATA_BASE = "data"
TIMETABLE_FILE = "timetable"
STOPS_FILE = "stops"
PATHS_FILE = "paths"
FILE_EXT = ".json"

In [98]:
ROUTE_API = lambda id: f"http://apicms.ebms.vn/businfo/getroutebyid/{id}"
TIMETABLE_API = lambda route: f"http://apicms.ebms.vn/businfo/gettimetablebyroute/{route}"
ROUTEVAR_API = lambda route: f"http://apicms.ebms.vn/businfo/getvarsbyroute/{route}"
STOPS_API = lambda id, varId: f"http://apicms.ebms.vn/businfo/getstopsbyvar/{id}/{varId}"
PATHS_API = lambda id, varId: f"http://apicms.ebms.vn/businfo/getpathsbyvar/{id}/{varId}"

def LoadData(route, source, api):
    source += FILE_EXT
    dir_path = os.path.join(os.getcwd(), DATA_BASE, str(route))
    file_path = os.path.join(os.getcwd(), DATA_BASE, str(route), source)

    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    if not os.path.exists(file_path):
        response = rq.get(api)
        with open(file_path, "w+", encoding="utf-8") as f:
            string = json.dumps(response.json(), ensure_ascii=False)
            f.write(string)
            
    return pd.read_json(file_path, encoding="utf-8")

In [99]:
class StationModel():
    def __init__(self, routeId, varId) -> None:
        src = STOPS_FILE + str(varId)
        api = STOPS_API(id=routeId, varId=varId)
        stops_df = LoadData(route=routeId, source=src, api=api)
        self.stops_df = stops_df[["StopId", "Lat", "Lng"]]

In [100]:
class BusModel():
    def __init__(self, id) -> None:
        self.stations = dict()
        self.paths_df = dict()
        timetables_df = LoadData(route=id, source=TIMETABLE_FILE, api=TIMETABLE_API(id))
        for varId in timetables_df["RouteVarId"]:
            self.stations[varId] = StationModel(routeId=id, varId=varId)
            self.paths_df[varId] = LoadData(route=id, source=PATHS_FILE, api=PATHS_API(id, varId=varId))

In [101]:
buses = list()
bus_ids = [8]
for id in bus_ids:
    buses.append(BusModel(id))

In [ ]:
HCM_NETWORK = "./data/hcm.gpkg"
if not os.path.exists(HCM_NETWORK):
    G = ox.graph_from_place("Ho Chi Minh City", simplify=False)
    ox.save_graph_geopackage(G, HCM_NETWORK, directed=True)
    
gdf_nodes = gpd.read_file(HCM_NETWORK, layer="nodes").set_index('osmid')
gdf_edges = gpd.read_file(HCM_NETWORK, layer="edges").set_index(['u', 'v', 'key'])
graph_attrs = {'crs': 'epsg:4326', 'simplified': True}
G = ox.graph_from_gdfs(gdf_nodes=gdf_nodes, gdf_edges=gdf_edges, graph_attrs=graph_attrs)

# fig, ax = ox.plot_graph(G, node_size=0)

In [132]:
HCM_NETWORK = "./data/hcm.osm"
utn = ox.settings.useful_tags_node
oxna = ox.settings.osm_xml_node_attrs
oxnt = ox.settings.osm_xml_node_tags
utw = ox.settings.useful_tags_way
oxwa = ox.settings.osm_xml_way_attrs
oxwt = ox.settings.osm_xml_way_tags
utn = list(set(utn + oxna + oxnt))
utw = list(set(utw + oxwa + oxwt))
ox.settings.all_oneway = True
ox.settings.useful_tags_node = utn
ox.settings.useful_tags_way = utw
if not os.path.exists(HCM_NETWORK):
    G = ox.graph_from_place("Ho Chi Minh City")
    ox.save_graph_xml(G, HCM_NETWORK)
    
G = ox.graph_from_xml(HCM_NETWORK)

# fig, ax = ox.plot_graph(G, node_size=0)

In [133]:
ox.distance.add_edge_lengths(G)
ox.add_edge_speeds(G)
ox.add_edge_travel_times(G)

In [164]:
paths = buses[0].paths_df[15]
paths["nodes"] = ox.nearest_nodes(G, X=paths["lng"], Y=paths["lat"])
paths["nodes"].tolist()

[2495870411,
 2495870411,
 6432690948,
 5755051489,
 4652534427,
 5755051488,
 366450563,
 5755153759,
 5755053133,
 6433515443,
 6433515443,
 6433515443,
 5755053133,
 6433515438,
 5816764351,
 6432751392,
 6432751391,
 6432751391,
 6432751391,
 6432751391,
 6432751391,
 6432751391,
 2040317180,
 6432749180,
 7473502484,
 4202267066,
 6449960893,
 4202267325,
 4202267325,
 4202267358,
 2040317166,
 6449960765,
 6449960735,
 6449960735,
 6449960735,
 6449960735,
 6449960735,
 6449960735,
 6450096913,
 6773057493,
 5814325558,
 6450096946,
 6450096946,
 6450096946,
 6450096946,
 6450096946,
 4616320836,
 6770224606,
 6643087494,
 1996745154,
 1996745814,
 1996653935,
 4609063287,
 4609063287,
 1996655330,
 1996655330,
 4609551766,
 4609551766,
 4609551765,
 4609551765,
 4609551765,
 1996655330,
 1996655291,
 1996655291,
 4609063287,
 366434468,
 5462393780,
 1996654802,
 1996654802,
 1996655233,
 6727805054,
 6727805054,
 6727805054,
 6727805054,
 6770923492,
 6727805037,
 6727805037,
 

In [201]:
paths = buses[0].stations[15].stops_df
paths["nodes"] = ox.nearest_nodes(G, X=paths["Lng"], Y=paths["Lat"])
def find_route(path):
    next_row = path.name + 1 if path.name + 1 < paths.shape[0] else path.name
    node_ids = ox.shortest_path(G, path["nodes"], paths.iloc[next_row]["nodes"])
    return node_ids
routes = paths.apply(find_route, axis="columns", result_type='reduce')
routes = routes.dropna().reset_index().drop("index", axis="columns")
routes

,0
0,"[2495870411.0, 4652534427, 5755051488, 3664505..."
1,"[6432751391.0, 5755153782, 6758003387, 2040317..."
2,"[6449960735.0, 3116882965, 6757924464, 6449960..."
3,"[6450096946.0, 4616320836, 366442410, 67703910..."
4,"[6727805054.0, 5816921541, 6770923492, 6783111..."
...,...
56,"[5778173023.0, 1958801546, 5778173032, 6778322..."
57,"[5778218361.0, 5778390214, 6616505160, 5778390..."
58,"[4831043101.0, 5762890412, 4831043103, 4831043..."
59,[3342113628.0]


In [215]:
route_map = ox.plot_route_folium(G, routes[0], node_size=0, show=False)
# for route in routes[1:]:
#     route_map = ox.plot_route_folium(G, route, route_map, node_size=0, show=False)

NetworkXError: Node [2495870411.0, 4652534427, 5755051488, 366450563, 5755153785, 6432690972, 5755153759, 5755153752, 5755053133, 6433515438, 6433515443.0] in sequence nbunch is not a valid node.